# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
sqrt_values = []
for sqrt_value in generator:
    print(sqrt_value)
    sqrt_values.append(sqrt_value)

print(sum(sqrt_values))


256


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




Add the dlt and duckdb libs

In [1]:
%%capture
!pip install "dlt[duckdb]" # Install dlt with all the necessary DuckDB dependencies

Verify work work

In [3]:
!dlt --version

dlt 0.4.4


Interactive with the data structure of the data which is a list and has an element of 1 dict

In [6]:
import dlt
import duckdb

# Define first generator, remember in this case yield is manually defining available data in the form of a dict (irl with will be from some api or db or dwh and yield will return a variable containing that one row of data)
# Whats really happening here? We are basically mimiciking the existance of data which we create from a simple range, and hold it in a dictionary, with different key:value pairs, representing one row of data in a table, keys = column names and their values.
# This for loop creates 5 dictionaries mimacking 5 rows of a table, this is a simple function which would normally return all 5 dictionaries but we would have to return them in some sort of container like for example in a list.
# However the yield function doesnt return anything unless the generator funct is iterated over with the next(), we instead can return just one row at a time and but need to run the next() on that fuctionc to get the next row without doing so there is no data, its just an iterable.
# So a generator is an iterable of functions, just like we would iterate thru a list, we can iterate thru a function that returns multiple items that we would normally have to return in full via a containter like object like a list or dict or tuple.
# The reason for this is memory management
# However understand that the dlt module handles the next() on a given generator either in a generator function() for itself or a variable assigned to a generator.

# Define first generator, remember in this case yield is manually defining available data in the form of a dict of one record at a time
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

#Save the generator object to a variable mostly for simplicity, remember that a genarator comprehension is basically list comprehension with () not []. This is confusing I asked my self why am I able to run a for loop on a function?
# thats because its no longer a function but an interable function (generator) which will only yield data if iterated upon with the next(). When u assign a generator comprehension to a variable, doesnt mean all data is being yielded, so think of it is as only way to get
#data is one at a time (to iterate over):
# for item in generator:
#   return item

#population_data_1 = (person for person in people_1())

pop_data_1 = people_1()

# for person in people_1():
#   print(person)

# print(next(population_data_1))
#print((person for person in people_1()))

#or
# (item for item in generator)


# print(population_data_1)
# print(population_data_1A)

# population_data_1 == population_data_1A
# print(next(population_data_1))
#{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
# print(next(population_data_1A))

#people_1()
#View elements in generator pop_1
#print(next(pop_1))

# print("\n")

# Define second generator, remember in this case yield is manually defining available data in the form of a dict of one record at a time
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# save the generator object, remember that a genarator comprehension is basicall list comprehension with () not []
#pop_2 = (person for person in people_2())
pop_data_2_ocp = people_2()

#View elements in generator pop_2
# print(next(pop_2))
# print(next(pop_2))

#define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='population_data')

# run the pipeline with default settings, and capture the outcome on the first generator pop_data_1 and save it to a table "population"
info = pipeline.run(pop_data_1, table_name="population", write_disposition="replace", primary_key="ID")

# # run the pipeline with default settings, and capture the outcome on the second generator pop_data_2 and save it to same table "population"
info = pipeline.run(pop_data_2_ocp, table_name="population", write_disposition="append", primary_key="ID")

# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
# print('Loaded tables: ')
display(conn.sql("show tables"))


# print("\n\n\n Population table below: Note the times are properly typed")
population_df = conn.sql("SELECT * FROM population").df()
display(population_df)

# print("\n\n\n Population table")
# passengers = conn.sql("SELECT * FROM rides__passengers").df()
# display(passengers)
# print("\n\n\n Stops table")
# stops = conn.sql("SELECT * FROM rides__stops").df()
# display(stops)


sum_of_all_ages_of_population = conn.sql("SELECT SUM(age) as summed_age FROM population").df()








┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ population          │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708667110.389884,Mf/0DZpIw1Foyw,None
1,2,Person_2,27,City_A,1708667110.389884,XbWCihdyl3R+Wg,None
2,3,Person_3,28,City_A,1708667110.389884,DBkjFiM7boX79g,None
3,4,Person_4,29,City_A,1708667110.389884,+moc0yowbWAB3A,None
4,5,Person_5,30,City_A,1708667110.389884,wRC8NAZ2DeXmPA,None
5,3,Person_3,33,City_B,1708667111.1789105,tEb9yY4Yq2y7tg,Job_3
6,4,Person_4,34,City_B,1708667111.1789105,8ahGuOJZv8a8LQ,Job_4
7,5,Person_5,35,City_B,1708667111.1789105,jdFndCaKWaFz7Q,Job_5
8,6,Person_6,36,City_B,1708667111.1789105,uTXpYGtx/7x7CQ,Job_6
9,7,Person_7,37,City_B,1708667111.1789105,9u48SjcarrGoVQ,Job_7


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [4]:
import dlt
import duckdb

# Define first generator, remember in this case yield is manually defining available data in the form of a dict of one record at a time
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


pop_data_1 = people_1()



# Define second generator, remember in this case yield is manually defining available data in the form of a dict of one record at a time
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


pop_data_2_ocp = people_2()





#define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='population_data2')

# run the pipeline with default settings, and capture the outcome on the first generator pop_data_1 and save it to a table "population"
info = pipeline.run(pop_data_1, table_name="population2", write_disposition="replace", primary_key="ID")

# run the pipeline with default settings, and capture the outcome on the second generator pop_data_2 and save it to same table "population"
info = pipeline.run(pop_data_2_ocp, table_name="population2", write_disposition="merge", primary_key="ID")

# Solution: First make sure that the following modules are installed:

In [7]:
# show the outcome

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
# print('Loaded tables: ')
display(conn.sql("show tables"))


# print("\n\n\n Population table below: Note the times are properly typed")
population_df2 = conn.sql("SELECT * FROM population2").df()
display(population_df2)





┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ population2         │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708667720.202021,gr8HzVOQj6BwoQ,None
1,2,Person_2,27,City_A,1708667720.202021,6djmraKn/E5o4w,None
2,8,Person_8,38,City_B,1708667720.9007652,xltgFJ9sF540HA,Job_8
3,4,Person_4,34,City_B,1708667720.9007652,OStSoNat26wz2w,Job_4
4,3,Person_3,33,City_B,1708667720.9007652,bdC9BwndbXweZQ,Job_3
5,6,Person_6,36,City_B,1708667720.9007652,yZFgX/7+KuX5Xw,Job_6
6,5,Person_5,35,City_B,1708667720.9007652,26s/VSIDKLN0IQ,Job_5
7,7,Person_7,37,City_B,1708667720.9007652,mIOKEfsniw6FMA,Job_7


In [8]:
sum_of_all_ages_of_population2 = conn.sql("SELECT SUM(age) as summed_age FROM population2").df()

In [10]:
display(sum_of_all_ages_of_population2)

,summed_age
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX